In [1]:
#!/usr/bin/env python3
"""
Python script to automatically generate MATLAB scripts for NEVIS convergence tests
with adjustable grid resolution and timestep parameters

Author: Auto-generated script
Date: 2025-09
"""

import os
import argparse
import sys
import numpy as np
from pathlib import Path

def format_scientific(value):
    if value >= 1:
        sci_str = f"{value:.0e}".replace('e+0', 'e').replace('e+', 'e')
    else:
        sci_str = f"{value:.0e}"
    
    return sci_str.replace('-', '_').replace('+', '')

def generate_convergence_script(moulin_input=0, Dx=125, kappa=1e-99, mu=1e2, V=1e7, theta=1e-3,
                               output_dir="./"):
    """
    Generate MATLAB script for convergence testing with specified grid and time resolution
    
    Parameters:
    -----------
    moulin_input : prescribed moulin input in m^3/s (default: 0)
    Dx : grid spacing in meters (default: 125)
    kappa : leakage coefficient (default: 1e-10)
    mu : viscosity (default: 1e2)
    V : lake volume (default: 1e7)
    output_dir : directory to save the generated script (default: current directory)
    """
    
    kappa_str = format_scientific(kappa)
    mu_str = format_scientific(mu)
    log10V = int(round(np.log10(V)))
    theta_str = format_scientific(theta).replace('.', '_')
    Dx_str = format_scientific(Dx).replace('.', '_')
    # Generate case name based on convergence parameters
    casename = f"n1d_E{moulin_input}_mu{mu_str}_kappa{kappa_str}_V1e{log10V}_theta{theta_str}_DX{Dx_str}_convergence"

    matlab_script = f"""
%% Script to run NEVIS regional model 
% This script is designed to run the NEVIS 1-dimensional model for an idealised ice sheet
% The surface runoff is set to 0, and the bedrock is linear with a slope of 0.01

% Author: Hanwen Zhang  
% Date: 2025-09
format compact

%% setup paths and directories
clear,clc
oo.root = './';                                % filename root
oo.code = '../nevis/src';                      % code directory   
oo.results = 'results';                        % path to the results folders
oo.dataset = 'nevis_regional';                 % dataset name     
oo.casename = '{casename}';                        % case name
                                            % casename
oo.fn = ['/',oo.casename];                     % filename (same as casename)
oo.rn = [oo.root,oo.results,oo.fn];            % path to the case results
oo.dn = [oo.root, 'data/', oo.dataset, '/'];   % path to the data
addpath(oo.code);                              % add path to code
mkdir(oo.rn);                                  % create directory for results 
oo.Tol_F = 1e-7;                               % tolerance for fsolve

%% parameters
% default parameters 
[pd,oo] = nevis_defaults([],oo);  

oo.evaluate_variables = 1;
oo.input_gaussian = 1;
oo.relaxation_term = 1;                         % 0 is alpha hb, 1 is alpha deltap hb
oo.initial_condition = 1;                       % 1 is default condition from 0365.mat, 0 is using steady-state drainage system, winter or summertime
oo.mean_perms = 1;
oo.modified_mean_perms = 0;
oo.display_residual =0;
pd.alpha_b = 0;                                 % relaxation rate (s^-1)
pd.kappa_b = {kappa};                             % relaxation coeff

% alter default parameters
pd.mu = {mu};                                  % water viscosity (Pa s)
pd.Ye = 8.8e9;                                  % Young's modulus (Pa)
pd.B = pd.Ye*(1e3)^3/(12*(1-0.33^2));           % bending stiffness (Pa m^3)
pd.c_e_reg2 = 0.00/1e3/9.81;                    % elastic sheet thickness [m/Pa]
pd.N_reg1 = 1e3;                                % effective pressure cut off for regularization of moulin input term [Pa]
pd.N_reg2 = 1e4; % 1e3                          % regularisation pressure for elastic sheet thickness  [Pa]
pd.u_b = 100/pd.ty;                             % sliding speed [m/s]
pd.sigma = 0e-3;                                % englacial void fraction
pd.h_r = 0.1;                                   % roughness height [m]
pd.l_r = 10;                                    % roughness length [m]
pd.l_c = 1000;                                  % sheet width contributing to conduit melting [m] default = 10 m
pd.k_s = 1e-3;                                  % sheet permeability constant
pd.tau_b = 60e3; %60 kPa                        % driving stress [Pa]
pd.melt = pd.G/pd.rho_w/pd.L;                   % geothermal heat derived basal melt [m/s]
pd.melt = 0*(pd.G+(pd.u_b*pd.tau_b))/pd.rho_w/pd.L;  % geothermal heat + frictional heating derived basal melt [m/s]
pd.meltinterior = ((pd.G+((100/pd.ty)*pd.tau_b))/pd.rho_w/pd.L)*1e3; % flux of basal melt up to the ~icedivide (200 km) [m2/s]
pd.E_lapse = 40/1000/pd.td/10^3;
pd.hb_reg1 = 1e-3;                              % regularizing thickness of blister [m]
pd.hb_reg2 = 1e-3;                              % regularizing thickness of blister [m]
pd.B_reg = 0;

% non-dimensionalise
ps = struct;
[ps,pp] = nevis_nondimension(pd,ps,oo); 

%% grid and geometry
L = 1e5;                               % length of the domain [m]
Dx = {Dx};
Nx = (L/Dx)+1;
x = linspace(0,(L/ps.x),Nx); 
y = linspace(0,(L/ps.x),1);            % 1-d grid of length 50km 
oo.yperiodic = 1;                      % oo.yperiodic = 1 necessary for a 1-d grid
oo.xperiodic = 0;
gg = nevis_grid(x,y,oo); 
b = 5e4/ps.z - {theta}*gg.nx*ps.x/ps.z;   % bed slope angle = 0.1
s = 200/ps.z + b;

%% mask with minimum ice thickness
H = max(s-b,0);
Hmin = 0/ps.z; 
nout = find(H<=Hmin);
gg = nevis_mask(gg,nout); 
gg.n1m = gg.n1;                                   % label all edge nodes as boundary nodes for pressure

%% label boundary nodes
gg = nevis_label(gg,gg.n1m);
oo.adjust_boundaries = 1;                         % enable option of changing conditions

%% initialize variables
[aa,vv] = nevis_initialize(b,s,gg,pp,oo);         % default initialisation
pd.k_f = 0.9;                                    % percent overburden (k-factor) 
vv.phi = aa.phi_a+pd.k_f*(aa.phi_0-aa.phi_a);     % initial pressure  k_f*phi_0
N = aa.phi_0-vv.phi;                              % N for initial cavity sheet size 
vv.hs = ((((pd.u_b*pd.h_r/pd.l_r)./((pd.u_b/pd.l_r)+(pd.K_c.*((ps.phi*N).^3)))))./ps.h); % initial cavity sheet size as f(N)

%% boundary conditions
aa.phi_b = max(aa.phi_0,aa.phi_a);                % prescribed boundary pressure at overburden or atmospheric (LAS 18 Nov. 2015)

%% moulins 
oo.keep_all_moulins = 0;
oo.random_moulins = 0;         
[pp.ni_m,pp.sum_m] = nevis_moulins([0.25*L/ps.x],[0/ps.x],gg,oo);     % one moulin at the lake location

%% supraglacial lakes
pp.x_l = [0.5*L/ps.x];                                          % x-coord of lakes
pp.y_l = [0];                                                   % y-coord of lakes
pp.V_l = [{V}/(ps.Q0*ps.t)];                                    % volume of lakes         
pp.t_drainage = [5*pd.td/ps.t];                                % time of lake drainages
pp.t_duration = [0.025*pd.td/ps.t];                             % duration of lake drainages
[pp.ni_l,pp.sum_l] = nevis_lakes(pp.x_l,pp.y_l,gg,oo);          % calculate lake catchments 

%% surface input
oo.surface_runoff = 0;                          % If set to 1 turns on surface runoff
                                                % If set to 0, moulin input is prescribed with a function
oo.RACMO_runoff = 0;                            % If set to 1, use RACMO input as below; 
                                                % If set to 0, use prescribed runoff function 
oo.distributed_input = 0;                       % If set to 1 turns on distributed input
                                                % If set to 0 , input is collected into moulins 
pp.input_function = @(t) {moulin_input}*(1-exp(-t/(30*pd.td/ps.t)))./(ps.m*ps.x^2);   % prescribed moulin input (m3/sec)

%% Timestep 
oo.dt = 1/24*pd.td/ps.t; 
oo.save_timesteps = 1; 
oo.save_pts_all = 1; 
oo.pts_ni = pp.ni_l;                             
oo.t_span = [(0.1:0.1:9.4)*pd.td/ps.t (9.5:0.005:10.5)*pd.td/ps.t (10.6:0.1:100)*pd.td/ps.t];
oo.t_span = [(0:0.005:20)*pd.td/ps.t];

%% save initial parameters
save([oo.rn, oo.fn],'pp','pd','ps','gg','aa','vv','oo');
[tt,vv,info] = nevis_timesteps(oo.t_span,vv,aa,pp,gg,oo);

%% expand/update variables
aa = nevis_inputs(vv.t,aa,vv,pp,gg,oo);
oo.evaluate_variables = 1; 
[vv2] = nevis_backbone(inf,vv,vv,aa,pp,gg,oo); 
vv2 = nevis_nodedischarge(vv2,aa,pp,gg,oo); 
save([oo.rn, oo.fn],'pp','pd','ps','gg','aa','oo','tt');
        """
    
    # Create output directory if it doesn't exist
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Write MATLAB script to file
    output_file = os.path.join(output_dir, f"{casename}.m")
    with open(output_file, 'w') as f:
        f.write(matlab_script)
    
    print(f"Generated convergence test script: {output_file}")
    
    return output_file

def batch_generate_convergence(convergence_combinations, output_dir="./generated_scripts/convergence_tests/"):
    """
    Generate multiple convergence test scripts with different grid/timestep combinations
    
    Parameters:
    -----------
    convergence_combinations : list of tuples
    output_dir : Directory to save all generated scripts
    """
    
    generated_files = []
    
    for i, (moulin_input, Dx, kappa, mu, V, theta) in enumerate(convergence_combinations):
        output_file = generate_convergence_script(
            moulin_input=moulin_input,
            Dx=Dx,
            kappa=kappa,
            mu=mu,
            V=V,
            theta=theta,
            output_dir=output_dir,
        )
        generated_files.append(output_file)
    
    return generated_files

def main():
    # Check if running in Jupyter
    if 'ipykernel' in sys.modules:
        # Running in Jupyter - use default parameters
        print("Running in Jupyter environment")
        return
    
    parser = argparse.ArgumentParser(description='Generate MATLAB scripts for NEVIS convergence tests')
    parser.add_argument('--batch', action='store_true', help='Generate a batch of convergence test scripts')
    parser.add_argument('--output_dir', type=str, default='./generated_scripts/convergence_tests/', help='Directory to save generated scripts')
    parser.add_argument('--moulin_input', type=float, default=0, help='Prescribed moulin input in m^3/s')
    parser.add_argument('--Dx', type=int, default=125, help='Grid spacing in meters')
    parser.add_argument('--kappa', type=float, default=1e-99, help='Leakage coefficient')
    parser.add_argument('--mu', type=float, default=1e2, help='Viscosity')
    parser.add_argument('--V', type=float, default=1e7, help='Volume')
    parser.add_argument('--theta', type=float, default=300, help='Drainage time')

    args = parser.parse_args()
    
    if args.batch:

        # Define convergence test parameter combinations
        convergence_combinations = [
            # (moulin_input, Dx, kappa, mu, V, theta)
            (0, 1000, 1e-99, 1e2, 1e8, 1e-2),
            (0, 500, 1e-99, 1e2, 1e8, 1e-2),
            (0, 250, 1e-99, 1e2, 1e8, 1e-2),
            (0, 125, 1e-99, 1e2, 1e8, 1e-2),
            (0, 62.5, 1e-99, 1e2, 1e8, 1e-2),
        ]
        
        print("Generating batch of convergence test scripts...")
        generated_files = batch_generate_convergence(convergence_combinations, args.output_dir)
        print(f"\nGenerated {len(generated_files)} convergence test scripts:")
        for file in generated_files:
            print(f"  {file}")
            
    else:
        # Generate single convergence test script
        output_file = generate_convergence_script(
        )
        print(f"Generated single convergence test script: {output_file}")

if __name__ == "__main__":
    main()

# Example usage for convergence testing
if __name__ == "__main__" or 'ipykernel' in sys.modules:
    # Define convergence test parameter combinations
    convergence_combinations = [
        # (moulin_input, Dx, kappa, mu, V, theta)
        (0, 1000, 1e-99, 1e0, 1e7, 1e-3),
        (0, 500, 1e-99, 1e0, 1e7, 1e-3),
        (0, 250, 1e-99, 1e0, 1e7, 1e-3),
        (0, 125, 1e-99, 1e0, 1e7, 1e-3),
        (0, 62.5, 1e-99, 1e0, 1e7, 1e-3),
        (0, 31.25, 1e-99, 1e0, 1e7, 1e-3),
        (0, 15.625, 1e-99, 1e0, 1e7, 1e-3),
        (0, 7.8125, 1e-99, 1e0, 1e7, 1e-3),
        (0, 1000, 1e-99, 1e-2, 1e7, 0e-2),
        (0, 500, 1e-99, 1e-2, 1e7, 0e-2),
        (0, 250, 1e-99, 1e-2, 1e7, 0e-2),
        (0, 125, 1e-99, 1e-2, 1e7, 0e-2),
        (0, 62.5, 1e-99, 1e-2, 1e7, 0e-2),
        (0, 31.25, 1e-99, 1e-2, 1e7, 0e-2),
    ]
        
    print("Generating convergence test scripts...")
    generated_files = batch_generate_convergence(convergence_combinations, "./generated_scripts/convergence_tests/")
    print(f"Generated {len(generated_files)} convergence test scripts")

Running in Jupyter environment
Generating convergence test scripts...
Generated convergence test script: ./generated_scripts/convergence_tests/n1d_E0_mu1e0_kappa1e_99_V1e7_theta1e_03_DX1e3_convergence.m
Generated convergence test script: ./generated_scripts/convergence_tests/n1d_E0_mu1e0_kappa1e_99_V1e7_theta1e_03_DX5e2_convergence.m
Generated convergence test script: ./generated_scripts/convergence_tests/n1d_E0_mu1e0_kappa1e_99_V1e7_theta1e_03_DX2e2_convergence.m
Generated convergence test script: ./generated_scripts/convergence_tests/n1d_E0_mu1e0_kappa1e_99_V1e7_theta1e_03_DX1e2_convergence.m
Generated convergence test script: ./generated_scripts/convergence_tests/n1d_E0_mu1e0_kappa1e_99_V1e7_theta1e_03_DX6e1_convergence.m
Generated convergence test script: ./generated_scripts/convergence_tests/n1d_E0_mu1e0_kappa1e_99_V1e7_theta1e_03_DX3e1_convergence.m
Generated convergence test script: ./generated_scripts/convergence_tests/n1d_E0_mu1e0_kappa1e_99_V1e7_theta1e_03_DX2e1_convergence.m